# Scenario Integration Test

Лаботоратория по искусственному интеллекту, Сбербанк. 

О чем: вызов сценариев с разными моделями.
В качестве датасета используется датасет MovieLens100K. 

## Содержание

1. [Импорты, создание спарк-сессии](#intro)
2. [Загрузка данных](#data-loader)
3. [Сценарии с разными моделями](#scenario)
3.1 [Получение сценария через фабрику](#get-scenario)
3.2 [Обучение сценария](#fit-scenario)

### Импорты, создание спарк-сессии <a name='intro'></a>

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os, sys

parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [4]:
import logging
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import pandas as pd
from sponge_bob_magic.datasets.movielens import MovieLens
from sponge_bob_magic.data_preparator.data_preparator import DataPreparator

from sponge_bob_magic.splitters import log_splitter
from sponge_bob_magic.splitters import user_log_splitter
from sponge_bob_magic import metrics

from sponge_bob_magic.models.pop_rec import PopRec
from sponge_bob_magic.models.als_rec import ALSRec
from sponge_bob_magic.models.knn_rec import KNNRec
from sponge_bob_magic.models.lightfm_rec import LightFMRec

from sponge_bob_magic.scenarios.main_scenario import MainScenario
from sponge_bob_magic.session_handler import  get_spark_session
from sponge_bob_magic.constants import DEFAULT_CONTEXT
from pyspark.sql.functions import lit

In [5]:
# отображение максимальной ширины колонок в pandas датафреймах
pd.options.display.max_colwidth = -1

In [6]:
spark = get_spark_session()
spark

## Загрузка данных <a name="data-loader"></a>

In [7]:
data = MovieLens("100k")
log = spark.createDataFrame(data.ratings).withColumn(
    "context", lit(DEFAULT_CONTEXT)
)
data.info()

ratings


,user_id,item_id,relevance,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116



users


,user_id,gender,age,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067



items


,item_id,title,release_date,imdb_url,unknown,...,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),1995-01-01,http://us.imdb.com/M/title-exact?Toy%20Story%20(1995),0,...,0,0,0,0,0
1,2,GoldenEye (1995),1995-01-01,http://us.imdb.com/M/title-exact?GoldenEye%20(1995),0,...,0,0,1,0,0
2,3,Four Rooms (1995),1995-01-01,http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995),0,...,0,0,1,0,0


## Сценарии с разными моделями <a name="scenario"></a>

### Получение сценария через фабрику <a name="get-scenario"></a>

In [8]:
pop_rec = PopRec()
als_rec = ALSRec()
knn_rec = KNNRec()
lightfm_rec = LightFMRec()

In [9]:
log_bydate_splitter = log_splitter.DateSplitter(
    test_start=datetime(2007, 1, 1),
    drop_cold_items=True,
    drop_cold_users=True
)
log_random_splitter = log_splitter.RandomSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)
log_cold_splitter = log_splitter.ColdUsersSplitter(
    test_size=0.3,
    drop_cold_items=True, 
    drop_cold_users=True
)
user_random_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True,
    drop_cold_users=True,
    shuffle=True,
    seed=1234
    
)
user_bydate_splitter = user_log_splitter.UserSplitter(
    item_test_size=0.3,
    user_test_size=500,
    drop_cold_items=True, 
    drop_cold_users=True,
    seed=1234
)

### Обучение сценария <a name="fit-scenario"></a>

In [10]:
results = None

In [11]:
scenario = MainScenario(
    splitter=user_random_splitter,
    recommender=lightfm_rec,
    criterion=metrics.HitRate,
    metrics={
        metrics.NDCG: [10,5,3],
        metrics.Precision: [10,5,3],
        metrics.MAP: [10,5,3],
        metrics.Recall: [10,5,3],
        metrics.Surprisal: [10,5,3],
    },
    fallback_rec=pop_rec,
)

In [12]:
popular_grid = {
    "alpha": {"type": "int", "args": [0, 10]},
    "beta": {"type": "int", "args": [0, 10]}
}
als_grid = {
    "rank": {"type": "discrete_uniform", "args": [10, 100, 10]}
}
lightfm_grid = {
    "rank": {"type": "int", "args": [10, 100]}
}
knn_grid = {
    "shrink": {"type": "discrete_uniform", "args": [10, 50, 10]},
    "num_neighbours": {"type": "discrete_uniform", "args": [0, 10, 1]},
}

In [13]:
best_params = scenario.research(
    lightfm_grid,
    log,
    k=10,
    n_trials=2
)

25-Feb-20 16:57:56, root, DEBUG: Деление лога на обучающую и тестовую выборку
25-Feb-20 16:58:16, root, DEBUG: Длина трейна и теста: (84013, 15944)
25-Feb-20 16:58:17, root, DEBUG: Количество пользователей в трейне и тесте: 943, 490
25-Feb-20 16:58:17, root, DEBUG: Количество объектов в трейне и тесте: 1644, 1355
25-Feb-20 16:58:17, root, DEBUG: Инициализация метрик
25-Feb-20 16:58:18, root, DEBUG: Обучение и предсказание дополнительной модели
25-Feb-20 16:58:18, root, DEBUG: Проверка датафреймов
25-Feb-20 16:58:18, root, DEBUG: Предварительная стадия обучения (pre-fit)
25-Feb-20 16:58:19, root, DEBUG: Среднее количество items у каждого user: 90
25-Feb-20 16:58:21, root, DEBUG: Основная стадия обучения (fit)
25-Feb-20 16:58:21, root, DEBUG: Проверка датафреймов
25-Feb-20 16:58:24, root, DEBUG: Количество items после фильтрации: 100
25-Feb-20 16:58:27, root, DEBUG: Пре-фит модели
25-Feb-20 16:58:27, root, DEBUG: -------------
25-Feb-20 16:58:27, root, DEBUG: Оптимизация параметров
25-Fe

In [14]:
results = pd.concat([scenario.study.trials_dataframe(), results], axis=0)

results

number                state     value             datetime_start  \
                                                                     
0  0      TrialState.COMPLETE  0.834694 2020-02-25 16:58:27.720338   
1  1      TrialState.COMPLETE  0.848980 2020-02-25 16:59:25.777942   

           datetime_complete params  \
                               rank   
0 2020-02-25 16:59:25.686747  85      
1 2020-02-25 17:00:21.365896  98      

                                                                user_attrs  \
                                                                       MAP   
0  {5: 0.40937585034013624, 10: 0.47331111458188685, 3: 0.365136054421769}   
1  {5: 0.4169852607709752, 10: 0.4816016995233465, 3: 0.37148526077097554}   

                                                                             \
                                                                  Precision   
0  {5: 0.22408163265306127, 10: 0.22877551020408163, 3: 0.2285714285714284}   
1  {5: 0.2240816326530613, 10: 0.23142857142857137, 3: 0.22653061224489787}   

                                                                              \
                                                                      Recall   
0  {5: 0.05078241926978814, 10: 0.09833609542586662, 3: 0.03055181220427518}   
1  {5: 0.04915484989722797, 10: 0.09999824471395656, 3: 0.02894789870830567}   

                                                                             \
                                                                  Surprisal   
0  {5: 0.1772261907289188, 10: 0.19152139244112618, 3: 0.17034874244381867}   
1  {5: 0.1880291680712521, 10: 0.19901286944125482, 3: 0.18197501573799374}   

                                                                              \
                                                                        nDCG   
0  {5: 0.2279346617247912, 10: 0.2370298684173544, 3: 0.23231141140584638}     
1  {5: 0.22958578874155391, 10: 0.23966974317411385, 3: 0.23281298322588304}   

  system_attrs  
       _number  
0  0            
1  1

### Получение рекомендаций <a name="predict-scenario"></a>

In [15]:
recs = scenario.production(
    best_params, 
    log,
    users=None, 
    items=None,
    k=10
)

25-Feb-20 17:00:24, root, DEBUG: Проверка датафреймов
25-Feb-20 17:00:25, root, DEBUG: Предварительная стадия обучения (pre-fit)
25-Feb-20 17:00:25, root, DEBUG: Основная стадия обучения (fit)
25-Feb-20 17:00:25, root, DEBUG: Построение модели LightFM
25-Feb-20 17:00:28, root, DEBUG: Проверка датафреймов
25-Feb-20 17:00:29, root, DEBUG: Выделение дефолтных юзеров
25-Feb-20 17:00:29, root, DEBUG: Выделение дефолтных айтемов


In [16]:
recs.show()

+-------+-------+--------------------+----------+
|user_id|item_id|           relevance|   context|
+-------+-------+--------------------+----------+
|     91|    300|  0.8993030786514282|no_context|
|     91|    511|  0.6741796135902405|no_context|
|     91|    187|  0.6400244832038879|no_context|
|     91|    510|   0.635736346244812|no_context|
|     91|    199| 0.43372300267219543|no_context|
|     91|    205| 0.31432321667671204|no_context|
|     91|    174| 0.19162911176681519|no_context|
|     91|    172| 0.18397298455238342|no_context|
|     91|    520| 0.16816245019435883|no_context|
|     91|    197| 0.12643621861934662|no_context|
|    152|    237|  0.9271017909049988|no_context|
|    152|     15|  0.9126777648925781|no_context|
|    152|    393|  0.7430113554000854|no_context|
|    152|    111|  0.6799671649932861|no_context|
|    152|     88|  0.6609490513801575|no_context|
|    152|     66|  0.5257515907287598|no_context|
|    152|    280| 0.13062837719917297|no_context|
